# APL Week 4

# Task 1

In [122]:
import networkx as nx

file=input("Enter the circuit file:")
netlist = open(file)
g = nx.DiGraph()
gateType = {}   #A dictionary to store node attributes
for line in netlist:
    line = line.strip()
    line = line.split() #reading through the file
    if(line[1].upper() != 'NOT' and line[1].upper() != 'INV'):  #adding edges 
        g.add_edges_from([(line[2],line[4])])
        g.add_edges_from([(line[3],line[4])])
        gateType[line[4]] = line[1]
    else:
        g.add_edges_from([(line[2],line[3])])
        gateType[line[3]] = line[1]
nl  = list(nx.topological_sort(g)) #Sorting the nodes in topological order
for i in nl:
    if i not in gateType:
        gateType[i] = "PI"
print(nl,gateType)

Enter the circuit file: c17.net


['N2', 'N7', 'N1', 'N3', 'N6', 'n_0', 'n_1', 'n_3', 'n_2', 'N22', 'N23'] {'N22': 'nand2', 'N23': 'nand2', 'n_3': 'nand2', 'n_2': 'nand2', 'n_0': 'nand2', 'n_1': 'nand2', 'N2': 'PI', 'N7': 'PI', 'N1': 'PI', 'N3': 'PI', 'N6': 'PI'}


In [120]:
def logicOp(a,b,gateType):       #Here I have created a function to calculate output for logical operation
    if(gateType.upper()=='AND2'):
        return (str(int(bool(int(a))) & int(bool(int(b)))))
    elif(gateType.upper()=='OR2'):
        return (str(int(bool(int(a))) | int(bool(int(b)))))
    elif(gateType.upper()=='NAND2'):
        return str(int(not (bool(int(a)) & bool(int(b)))))
    elif(gateType.upper()=='NOR2'):
        return str(int(not (bool(int(a)) | bool(int(b)))))
    elif(gateType.upper()=='XOR2'):
        return (str(int(bool(int(a))) ^ int(bool(int(b)))))
    elif(gateType.upper()=='XNOR2'):
        return str(int(not (bool(int(a)) ^ bool(int(b)))))
    pass

In [121]:
def print_fin(values, values2):  # A function to print values in required format
    
    keys = list(values.keys())
    keys.sort()
    values = {i: values[i] for i in keys}
    
    for i in values:
        print('{:<5}'.format(i), end=' ') 
    print()
    
    for val in values2:
        keys = list(val.keys())
        keys.sort()
        val = {i: val[i] for i in keys}
        for j in val:
            print(val[j], end='     ') 
        print()

# Task 2

## 1. Topologically Ordered evaluation

In [111]:
def circSolve(net,states):
    Error1 = "Circuit is in combinational loop"
    netlist = open(net)
    g = nx.DiGraph()
    gates = {}
    for line in netlist:
        line = line.strip()
        line = line.split()
        if(line[1].upper() != 'NOT' and line[1].upper() != 'INV'):
            g.add_edges_from([(line[2],line[4])])
            g.add_edges_from([(line[3],line[4])])
            gates[line[4]] = line[1]
        else:
            g.add_edges_from([(line[2],line[3])])
            gates[line[3]] = line[1]
    try:
        topology = list(nx.topological_sort(g))
    except:
        return Error1 #Topological sort fails for combinational loops
    for i in topology:
        if i not in gates:
            gates[i] = "PI"
    inputs = open(states)
    inp = []
    for line in inputs:
        line = line.strip()
        line = line.split()
        inp.append(line)
    value_table=[] #this is a list of dictionaries to store outputs for all states
    for i in range(1,len(inp)):
        value = {} #this dictionary will store output at each node for a given state
        for j in range(len(inp[0])):
            value[inp[0][j]] = inp[i][j] #adding values of primary inputs to the dictionary
        for node in topology: #going through the circuit topologically, evaluating output at each node and adding it to the dictionary
            if(gates[node] != 'PI'):
                pred = list(g.predecessors(node))
                if(gates[node].upper()== 'NOT' or gates[node].upper() == 'INV'):
                    value[node] = str(int(not bool(int(value[pred[0]]))))
                else:
                    value[node] = logicOp(value[pred[0]],value[pred[1]],gates[node])
        value_table.append(value)
    print_fin(value,value_table)
                
    pass
file=input("Enter the circuit file:")
fhand = input("Enter the input file:")
circSolve(file,fhand)


Enter the circuit file: c17.net
Enter the input file: c17.inputs


N1    N2    N22   N23   N3    N6    N7    n_0   n_1   n_2   n_3   
0     1     1     1     0     0     0     1     1     1     0     
0     0     0     0     1     0     0     1     1     1     1     
1     0     0     0     0     0     0     1     1     1     1     
0     0     0     0     1     1     1     1     0     1     1     
1     1     1     0     1     1     1     0     0     1     1     
1     1     1     1     1     0     0     0     1     1     0     
1     1     1     0     1     1     0     0     0     1     1     
1     1     1     1     0     0     0     1     1     1     0     
0     1     1     1     1     0     1     1     1     0     0     
0     0     0     0     1     1     0     1     0     1     1     


### Timing the code

In [109]:
%timeit circSolve("c432.net","c432.inputs")

26 ms ± 1.29 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [110]:
%timeit circSolve("c17.net","c17.inputs")

232 µs ± 13.5 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## 2. Event-Driven evaluation

In [123]:
def circSolve_2(net,states):
    Error1 = "Circuit is in combinational loop"
    netlist = open(net)
    g = nx.DiGraph()
    gates = {}
    for line in netlist:
        line = line.strip()
        line = line.split()
        if(line[1].upper() != 'NOT' and line[1].upper() != 'INV'):
            g.add_edges_from([(line[2],line[4])])
            g.add_edges_from([(line[3],line[4])])
            gates[line[4]] = line[1]
        else:
            g.add_edges_from([(line[2],line[3])])
            gates[line[3]] = line[1]
    try:
        topology = list(nx.topological_sort(g))
    except:
        return Error1
    for i in topology:
        if i not in gates:
            gates[i] = "PI"
    inputs = open(states)
    inp = []
    for line in inputs:
        line = line.strip()
        line = line.split()
        inp.append(line)
    queue_2 = []  #Creating a queue using lists
    value = {} #this will hold the output at each node for a given state
    for i in topology:
        value[i] = 0  #making all nodes zero in the value dictionary in the beginning
    ky = list(value.keys())
    ky.sort()
    value = {i: value[i] for i in ky}  #sorting all nodes in alphabetical order and printing them with formatting
    for i in value:
        print('{:<5}'.format(i), end=' ')
    print()
    for i in range(1,len(inp)):
        queue_2 = []
        for j in range(len(inp[i])):
            if(value[inp[0][j]] != inp[i][j]): #when we go from one state to another, we check which primary input has changed
                value[inp[0][j]] = inp[i][j]
                for t in list(g.successors(inp[0][j])):
                    if(t not in queue_2):
                        queue_2.append(t) #and then we add its successsors to the queue
        while(len(queue_2)>0):
            n = queue_2[0] #now we start evaluating the queue
            pred = list(g.predecessors(n))
            if(gates[n].upper() == 'NOT' or gates[n].upper() == 'INV'):
                value[n] = str(int(not bool(int(value[pred[0]]))))
            else:
                value[n] = logicOp(value[pred[0]],value[pred[1]],gates[n])
            queue_2.pop(0)
            for t in list(g.successors(n)):
                if(t not in queue_2):
                    queue_2.append(t)
        k = list(value.keys())
        for one in k:
            print(value[one],end='     ')
        print()
    pass
file=input("Enter the circuit file:")
fhand = input("Enter the input file:")
circSolve_2(file,fhand)

Enter the circuit file: c17.net
Enter the input file: c17.inputs


N1    N2    N22   N23   N3    N6    N7    n_0   n_1   n_2   n_3   
0     1     1     1     0     0     0     1     1     1     0     
0     0     0     0     1     0     0     1     1     1     1     
1     0     0     0     0     0     0     1     1     1     1     
0     0     0     0     1     1     1     1     0     1     1     
1     1     1     0     1     1     1     0     0     1     1     
1     1     1     1     1     0     0     0     1     1     0     
1     1     1     0     1     1     0     0     0     1     1     
1     1     1     1     0     0     0     1     1     1     0     
0     1     1     1     1     0     1     1     1     0     0     
0     0     0     0     1     1     0     1     0     1     1     


### Timing the code

In [106]:
%timeit circSolve_2("c432.net","c432.inputs")

501 ms ± 18.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [105]:
%timeit circSolve_2("c17.net","c17.inputs")

293 µs ± 10.1 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


# Task 3

After timing both the functions we can see that th event-driven approach is taking longer than the topologically sorted evaluation for inputs with large number of gates and states. Both the approaches take similar amount of time when we compare them for smaller circuits. This is because I have used list to visualise queues here. 
In lists, everytime we pop an element in the beginning, we have to shift all the other elements by one which gives us a time complexity of O(n). One way to reduce the time is by using the collections.deque and queue.Queue libraries.